In [1]:
import numpy as np
from sympy import symbols, Matrix, simplify, pprint
import matplotlib.pyplot as plt
from pyscf import gto, scf
import sys
sys.path.append('/workspaces/bachelor-project-bridging-profile-Mattice-Criel/projects/Project-MatticeCriel/Benzene/rhf')
import plottingsystem as pts

In [2]:
K = 1.75
alpha_C = -11.4
alpha_N = -11.4

In [3]:
n = 4
H = np.zeros((n, n))
print(H)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [4]:
for i in range(n):
    H[i, i] = alpha_C
    if i == 0:
        H[i, i] = alpha_N

print(H)

[[-11.4   0.    0.    0. ]
 [  0.  -11.4   0.    0. ]
 [  0.    0.  -11.4   0. ]
 [  0.    0.    0.  -11.4]]


In [5]:
cyclobutadiene = """  C    0.7322530   -0.6646198    0.0000000
  C    0.7322530    0.6646198    0.0000000
  C   -0.7322530    0.6646198   -0.0000000
  C   -0.7322530   -0.6646198   -0.0000000
  H    1.5215298   -1.4088346    0.0000000
  H    1.5215298    1.4088346    0.0000000
  H   -1.5215298    1.4088346   -0.0000000
  H   -1.5215298   -1.4088346   -0.0000000
  """

carbon_coordinates  = pts.taking_carbon_coordinates(cyclobutadiene)
print(carbon_coordinates)


benzene_molecule_with_H = gto.M(atom=cyclobutadiene, basis='sto-3g')#'ccpvdz''sto-6g'

pz_indices = [i for i, label in enumerate(benzene_molecule_with_H.ao_labels()) if 'C 2pz' in label]
print(pz_indices)

rhf = scf.RHF(benzene_molecule_with_H)
rhf_energy = rhf.kernel()

[[ 0.732253  -0.6646198  0.       ]
 [ 0.732253   0.6646198  0.       ]
 [-0.732253   0.6646198 -0.       ]
 [-0.732253  -0.6646198 -0.       ]]
[4, 9, 14, 19]


converged SCF energy = -151.727655705318


In [6]:
S = rhf.get_ovlp()[[4, 9, 14, 19]][:, [4, 9, 14, 19]]
print(S)

[[1.         0.2418438  0.06839926 0.18928155]
 [0.2418438  1.         0.18928155 0.06839926]
 [0.06839926 0.18928155 1.         0.2418438 ]
 [0.18928155 0.06839926 0.2418438  1.        ]]


In [7]:
for i in range(n):
    for j in range(n):
        if i != j:
            H[i, j] = H[j, i] = K*S[i,j]*(H[i,i]+H[j,j])/2
print(H)

[[-11.4         -4.82478378  -1.36456514  -3.77616695]
 [ -4.82478378 -11.4         -3.77616695  -1.36456514]
 [ -1.36456514  -3.77616695 -11.4         -4.82478378]
 [ -3.77616695  -1.36456514  -4.82478378 -11.4       ]]


In [8]:
# ==> Inspecting S for AO orthonormality <==
def controleer_orthonormaliteit(matrix):
    is_ortho = np.allclose(np.eye(len(matrix[0])), matrix)
    if is_ortho:
        print("De matrix is orthonormaal.")
    else:
        print("De matrix is niet orthonormaal.")
controleer_orthonormaliteit(S)

De matrix is niet orthonormaal.


In [9]:
# ==> Construct AO orthogonalization matrix A <==
from scipy.linalg import fractional_matrix_power
A = fractional_matrix_power(S, -0.5)
A = np.asarray(A)
ASA = A@S@A

# Check orthonormality
controleer_orthonormaliteit(ASA)

De matrix is orthonormaal.


In [10]:
H_p = A @ H @ A 
epsilon, C_p = np.linalg.eigh(H_p)

print(epsilon, C_p)
C = A @ C_p
print(C)
eigenvalues = A@epsilon
print(eigenvalues)
for i in range(n):
    C[:,i] = (np.dot(C[:,i], C[:,i]))**(-1/2) * C[:,i]

print(C)

[-14.24819293 -11.26241464 -10.22346354  -6.53347701] [[-0.5 -0.5  0.5  0.5]
 [-0.5 -0.5 -0.5 -0.5]
 [-0.5  0.5 -0.5  0.5]
 [-0.5  0.5  0.5 -0.5]]
[[-0.408313   -0.50400691  0.53329321  0.62633537]
 [-0.408313   -0.50400691 -0.53329321 -0.62633537]
 [-0.408313    0.50400691 -0.53329321  0.62633537]
 [-0.408313    0.50400691  0.53329321 -0.62633537]]
[-12.73800806  -8.93228992  -8.70164888  -4.14483179]
[[-0.5 -0.5  0.5  0.5]
 [-0.5 -0.5 -0.5 -0.5]
 [-0.5  0.5 -0.5  0.5]
 [-0.5  0.5  0.5 -0.5]]


In [11]:
pts.plot_MOs(eigenvalues, C, carbon_coordinates, 'Cyclobutadiene', 'Extended Hückel Theory')

TypeError: plot_MOs() missing 1 required positional argument: 'method'